In [ ]:
import numpy as np
np.random.seed(0)

Primeiro vamos criar uma classe que vai representar uma aresta direcional. Essa classe não é necessária para implementar um grafo (poderíamos simplesmente usar tuplas com os índices dos nós), no entanto facilita a leitura do código.

Ela vai armezar apenas os dois nós e o peso. Ela pode ser considerada uma aresta não-direcional também, a única diferença é que estamos dando nomes diferentes para os dois nós.

In [ ]:
class Edge:
    """Uma aresta direcional com nós de "saída", "chegada" e peso."""
    def __init__(self, u, v, w = 1):
        self.src = u
        self.dest = v
        self.w = w

    def __str__(self):
        return f"{self.src} -> {self.dest}"

Caso seja necessário, também seria possível criar uma classe para os nós, para salvar determinados atributos.



In [ ]:
n_nodes = 6
n_edges = 12

selected_edges = []

for _ in range(n_edges):
    u = int(np.random.randint(n_nodes))
    v = int(np.random.randint(n_nodes))

    while u == v or ((u, v) in selected_edges) or ((v, u) in selected_edges):
        u = int(np.random.randint(n_nodes))
        v = int(np.random.randint(n_nodes))
    
    selected_edges.append((u, v))


## Adjacency Matrix



É o método de representação de grafos mais comum quando tratamos da representação matemática deles (facilita a nomenclatura, fórmulas).

Um grafo de $n$ nós é criado através de uma matriz $A$ de tamanho $n \times n$. $A[i, j]$ registra o peso da aresta presente entre os vértices $i$, $j$, e pode ser $0$ caso não exista a origem.

**Adicionar novo nó**: É necessário criar uma nova matriz aumentando o tamanho do matriz.

**Adicionar nova aresta**: Basta atualizar o valor da posição dessa aresta.

**Remover nó:**: É necessário criar uma nova matriz diminuindo o tamanho da matriz. Uma maneira também é só zerar a coluna e a linha desse nó.

**Remover aresta:** Basta atualizar o valor da posição dessa aresta.

In [ ]:
class AdjacencyMatrix:
    def __init__(self, n):
        self.n = n
        self.adj = [[None]*n for _ in range(n)]

    def __str__(self):
        s = ""
        for u in range(self.n):
            for v in range(self.n):
                if self.adj[u][v]:
                    s += f"{u} -> {v}, "
        return s[:-1]

    def new_edge(self, u, v, w = 1):
        edge = Edge(u, v, w)
        self.adj[u][v] = edge

    def get_edge(self, u, v):
        if not self.adj[u][v] is None:
            return self.adj[u][v]
        
        raise ValueError("Edge not in graph")

    def remove_edge(self, u, v):
        if self.adj[u][v]:
            self.adj[u][v] = None
        
        raise ValueError("Edge not in graph")
    
    def get_neighbors(self, u):
        neighbors = []
        for v in range(self.n):
            if self.adj[u][v]:
                neighbors.append(v)
            elif self.adj[v][u]:
                neighbors.append(v)
        
        return neighbors

In [ ]:
G = AdjacencyMatrix(n_nodes)

for (u, v) in selected_edges:
    G.new_edge(u, v)

e = G.get_edge(2, 3)
N = G.get_neighbors(2)
print("Vizinhos do 2:", N)
print(G)

Vizinhos do 2: [0, 1, 3]
0 -> 2, 0 -> 3, 0 -> 4, 1 -> 0, 2 -> 1, 2 -> 3, 3 -> 4, 4 -> 1, 4 -> 5, 5 -> 0, 5 -> 1, 5 -> 3,


👍 A vantagem da AdjacencyMatrix é que atualizar arestas é muito rápido.

👎 A desvantagem é que adicionar/remover nós pode ser custoso, e que nós armazenamos bastante memória quando o grafo é esparso (poucas arestas entre as possíveis existem).

## EdgeList

A EdgeList como o próprio nome já diz é uma estrutura que armazena as arestas como uma lista. Quando um grafo é esparso (com poucas arestas), ela ocupa menos espaço para armazenar todas as arestas.

**Adicionar novo nó**: Ela não armazena os nós, só se eles tiverem participando de alguma aresta. 

**Adicionar nova aresta**: Basta adicionar um novo elemento na lista de arestas.

**Remover nó:**: É necessário percorrer todas as arestas e verificar se o nó está presente.

**Remover aresta:** É *necessário* percorrer todas as arestas e verificar se essa é a aresta que será removida.

In [ ]:
class EdgeList:
    """Grafo construido através de uma EdgeList. Como o próprio nome já diz, é uma lista
    que armazena cada uma das arestas."""
    def __init__(self):
        self.edges = []
    
    def __str__(self):
        return ", ".join([f"{edge.src} -> {edge.dest}" for edge in self.edges])

    def new_edge(self, u, v, w = 1):
        edge = Edge(u, v, w)
        self.edges.append(edge)

    def get_edge(self, u, v):
        for edge in self.edges:
            if (u == edge.src and v == edge.dest):
                return edge
        
        raise ValueError("Edge not in graph")

    def remove_edge(self, u, v):
        for edge in self.edges:
            if (u == edge.src and v == edge.dest):
                self.edges.remove(edge)
                return edge
        
        raise ValueError("Edge not in graph")
    
    def get_neighbors(self, u):
        neighbors = []
        for edge in self.edges:
            if u == edge.src:
                neighbors.append(edge.dest)
            elif u == edge.dest:
                neighbors.append(edge.src)
        
        return neighbors


In [ ]:
G = EdgeList()

for (u, v) in selected_edges:
    G.new_edge(u, v)

e = G.get_edge(2, 3)
N = G.get_neighbors(2)
print("Vizinhos do 2:", N)
print(G)

Vizinhos do 2: [0, 3, 1]
5 -> 0, 5 -> 1, 0 -> 2, 2 -> 3, 2 -> 1, 4 -> 1, 0 -> 3, 0 -> 4, 3 -> 4, 5 -> 3, 4 -> 5, 1 -> 0


👍 A vantagem da EdgeList é que adicionar novas arestas é muito rápido, e que usa menos memória.

👎 A desvantagem é que encontrar uma aresta e remover arestas/nós são tarefas  demoradas.

## AdjacencyList


É uma terceira versão em que para cada nós salvamos uma lista das arestas que ele possui. Seja $A$ essa lista maior, a posição $A[i]$ salva as arestas do nó $i$ (apenas as que existem).

**Adicionar novo nó**: É necessário adicionar um novo elemento na lista de vértices.

**Adicionar nova aresta**: Adicionamos uma nova aresta na lista de arestas do nó.

**Remover nó:** Removemos uma aresta da lista de arestas do nó.

**Remover aresta:** Removemos uma aresta da lista de arestas de um nó.

In [ ]:
class AdjacencyList:
    def __init__(self, n):
        self.n = n
        self.vertex = [[] for _ in range(n)]

    def __str__(self):
        s = ""
        for v in range(self.n):
            for edge in self.vertex[v]:
                s += f" {edge.src} -> {edge.dest},"
        return s[:-1]

    def new_edge(self, u, v, w = 1):
        edge = Edge(u, v, w)
        self.vertex[u].append(edge)

    def get_edge(self, u, v):
        for edge in self.vertex[u]:
            if edge.dest == v:
                return edge
        
        raise ValueError("Edge not in graph")

    def remove_edge(self, u, v):
        for edge in self.vertex[u]:
            if edge.dest == v:
                self.vertex[u].remove(edge)
                return
        
        raise ValueError("Edge not in graph")
    
    def get_neighbors(self, u):
        neighbors = []
        for edge in self.vertex[u]:
            neighbors.append(edge.dest)
        
        #for v in range(self.n):
        #    for edge in self.vertex[v]:
        #        if edge.dest == u:
        #            neighbors.append(edge.src)
        return neighbors

In [ ]:
G = AdjacencyList(n_nodes)

for (u, v) in selected_edges:
    G.new_edge(u, v)

e = G.get_edge(2, 3)
N = G.get_neighbors(2)
print("Vizinhos do 2:", N)
print(G)

Vizinhos do 2: [3, 1]
 0 -> 2, 0 -> 3, 0 -> 4, 1 -> 0, 2 -> 3, 2 -> 1, 3 -> 4, 4 -> 1, 4 -> 5, 5 -> 0, 5 -> 1, 5 -> 3


👍 A vantagem da AdjacencyList é que adicionar/remover arestas é eficiente, adicionar nós também é eficiente.

👎 A desvantagem é quando o número de arestas para um nó em específico é muito grande, as operações se tornam mais lentas, e remover nós é custoso.


## Representação dos nós

Todas as implementações acima utilizaram os vértices como inteiros $\{1, \dots, n \}$. No entanto, geralmente identificamos os nós com outros "nomes". Uma maneira de lidar com isso é utilizar dicionários que mapeiam do nome para o índice.

In [ ]:
class AdjacencyMatrix:
    def __init__(self, vertex_names, n):
        self.vertex = dict([(name, i) for (i, name) in enumerate(vertex_names)])
        self.n = n
        self.adj = [[None]*n for _ in range(n)]

    def __str__(self):
        s = ""
        for u in range(self.n):
            for v in range(self.n):
                if self.adj[u][v]:
                    s += f"{u} -> {v}, "
        return s[:-1]

    def new_edge(self, u_name, v_name, w = 1):
        u = self.vertex[u_name]
        v = self.vertex[v_name]
        edge = Edge(u, v, w)
        self.adj[u][v] = edge

    def get_edge(self, u_name, v_name):
        u = self.vertex[u_name]
        v = self.vertex[v_name]
        if self.adj[u][v]:
            return self.adj[u][v]
        
        raise ValueError("Edge not in graph")

    def remove_edge(self, u_name, v_name):
        u = self.vertex[u_name]
        v = self.vertex[v_name]
        if self.adj[u][v]:
            self.adj[u][v] = None
        
        raise ValueError("Edge not in graph")